In [3]:
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
from MetaWorld.searchTest.utils import MyEnv, ToyExpertModel, sample_expert_transitions, benchmark_policy, LearnWrapper, train_policy, LearnWrapper, parse_sampled_transitions
import torch
import zipfile
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
import importlib
import numpy as np
from RlBaselines3Zoo import enjoy
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.ppo import MlpPolicy

from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper

from stable_baselines3.common.policies import MultiInputActorCriticPolicy, ActorCriticPolicy, BaseModel
from stable_baselines3.common.torch_layers import (
    BaseFeaturesExtractor,
    CombinedExtractor,
    FlattenExtractor,
    NatureCNN,
    create_mlp,
    get_actor_critic_arch,
)
from sb3_contrib.tqc.tqc import TQC
from sb3_contrib.tqc.policies import MultiInputPolicy
import numpy as np
import torch as th

model = enjoy.main(inpt_args="--algo tqc --env FetchPickAndPlace-v1 --folder /home/hendrik/Documents/master_project/Code/RlBaselines3Zoo/rl-trained-agents -n 300 --ret_model True")

Loading latest experiment, id=1
Loading /home/hendrik/Documents/master_project/Code/RlBaselines3Zoo/rl-trained-agents/tqc/FetchPickAndPlace-v1_1/FetchPickAndPlace-v1.zip


/home/hendrik/anaconda3/envs/tfTest/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [4]:
model.policy.predict(model.env.reset())

(array([[-0.9656587,  0.9957423,  0.5810138, -0.6076181]], dtype=float32),
 None)

In [5]:
from utilsMW.dataLoaderMW import TorchDatasetMW
path = '/home/hendrik/Documents/master_project/LokalData/metaworld/pick-place/training_data/'

In [6]:
data = TorchDatasetMW(path=path, device='cuda', n=100)

In [7]:
data.data[0,0,:5]

tensor([ 0.0051,  0.6004,  0.1936, -0.0963,  0.6872], device='cuda:0')

In [8]:
data.data[0,10,:5]

tensor([-0.0715,  0.6517,  0.1481, -0.0964,  0.6872], device='cuda:0')

In [9]:
data.label.shape

torch.Size([100, 200, 4])

In [10]:
#env = SuperMyGymWrapper(tag = None, bo=env).make_wrapper()
def sample_expert_transitions():
    expert = model

    print("Sampling expert transitions.")
    rollouts = rollout.rollout(
        expert,
        DummyVecEnv([lambda: RolloutInfoWrapper(model.env.envs[0])]),
        rollout.make_sample_until(min_timesteps=None, min_episodes=10),
        unwrap=True,
        exclude_infos=False,
    )
    return rollout.flatten_trajectories(rollouts)

#tp = '/home/hendrik/Documents/master_project/LokalData/ImitationLearning/transitions_rew_test'

transitions = sample_expert_transitions()
#torch.save(transitions, tp)
#transitions = torch.load(tp)

Sampling expert transitions.


In [3]:
from MetaWorld.searchTest.utils import parse_sampled_transitions, VecExtractor

In [4]:
extractor = VecExtractor(CombinedExtractor, env=model.env)

In [ ]:
bc_policy = MultiInputActorCriticPolicy(observation_space=model.observation_space, action_space=model.action_space, lr_schedule=lambda a: 1, features_extractor_class=CombinedExtractor, net_arch=[512, 512, 512])

In [ ]:
sac_policy = SAC(policy=MultiInputPolicy, env=model.env, policy_kwargs=dict(features_extractor_class=CombinedExtractor, net_arch=[512, 512, 512]), learning_rate=1e-3)

In [ ]:
asd = PPO(policy=MultiInputPolicy, env=model.env, policy_kwargs=dict(features_extractor_class=CombinedExtractor, net_arch=[512, 512, 512]), learning_rate=1e-3)

In [ ]:
env = model.env

In [ ]:
bc_policy = MultiInputPolicy(observation_space=env.observation_space, action_space=env.action_space, lr_schedule=lambda a:1, net_arch=[512,512,512], features_extractor_class=CombinedExtractor, )

In [ ]:
sd = bc_policy.state_dict()

In [ ]:
sac_policy.policy.load_state_dict(sd)

In [ ]:
env = model.env
bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    policy=bc_policy,
)

print("Training a policy using Behavior Cloning")

In [ ]:
train_policy(
    trainer=bc_trainer,
    learn_fct=bc_trainer.train,
    val_env=model.env,
    logname='BC MultiInputPolicy 100',
    path='/home/hendrik/Documents/master_project/LokalData/ImitationLearning/',
    n_epochs=1000,
    n_steps=50,
    eval_epochs=100,
    step_fct=lambda i: i+1
)

In [ ]:
sac_policy = SAC(policy=MultiInputPolicy, env=model.env, policy_kwargs=dict(features_extractor_class=CombinedExtractor, net_arch=[512, 512, 512]), learning_rate=1e-3)

In [ ]:
sac_policy.policy.load_state_dict(bc_policy.state_dict())

In [ ]:
benchmark_policy(sac_policy, '/home/hendrik/Documents/master_project/LokalData/ImitationLearning/', 'benchmark SAC', eval_epochs=200, val_env=model.env, stepid=1, best_reward=-1, save_model=False)

In [ ]:
bc_trainer.train(n_epochs=100)

In [ ]:
#path_to_zip_file = '/home/hendrik/Documents/master_project/Code/rl-baselines3-zoo/rl-trained-agents/her/FetchPickAndPlace-v1_1/FetchPickAndPlace-v1.zip'
path_to_zip_file = '/home/hendrik/Documents/master_project/Code/rl-baselines3-zoo/logs/tqc/FetchPickAndPlace-v1_1/FetchPickAndPlace-v1.zip'
path_to_zip_file = '/home/hendrik/Documents/master_project/Code/rl-baselines3-zoo/rl-trained-agents/tqc/FetchPickAndPlace-v1_1/FetchPickAndPlace-v1.zip'
directory_to_extract_to = '/tmp/sbtest'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
policy = torch.load('/tmp/sbtest/policy.pth')

In [ ]:
train_policy(
    trainer=model, 
    learn_fct=learn_wrapper.train, 
    val_env=val_env, 
    logname='HER Test', 
    path='/home/hendrik/Documents/master_project/LokalData/nISB/',
    n_epochs=1000,
    n_steps=20,
    eval_epochs=100)